In [2]:
%pip install plotly

     |████████████████████████████████| 13.2 MB 1.9 MB/s eta 0:00:01
  Created wheel for retrying: filename=retrying-1.3.3-py3-none-any.whl size=11429 sha256=ebb5c09198510a0d1649f4593d14ff9560cc05d3c5502639958b5f6491503aa8
  Stored in directory: /Users/carlos.vega/Library/Caches/pip/wheels/c4/a7/48/0a434133f6d56e878ca511c0e6c38326907c0792f67b476e56
Successfully built retrying
You should consider upgrading via the '/Users/carlos.vega/dev/biokc-paper/env/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [16]:
#init_notebook_mode(connected=True)
%matplotlib inline
import matplotlib, plotly, fileinput
import matplotlib.pyplot as plt
import pandas as pd 
from collections import Counter

from plotly.offline import plot, iplot, init_notebook_mode
import plotly.graph_objs as go
init_notebook_mode(connected=True)

In [43]:
dates = []
for line in fileinput.input('dates.txt'):
    line = line.rstrip()
    
    #ignore lines starting with # or ending with ?
    if line[0] == '#' or line[-1] == '?':
        continue
    
    _, date = line.split('|') # split lines
    date = date.replace('', '') # remove spaces
    dates.append(pd.Timestamp(date))
    
dates = sorted(list(set(dates))) #Multiple elections happening on the same date are considered as one.

In [44]:
colors = ['#eebcbc', '#eedeab', '#c8ecb5', '#bcc2f2', '#e8b6ee', '#b25a86', '#546e85']

#all data together
d = Counter([t.day_name() for t in dates])
labels = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
all_values = [d[l] for l in labels]

empty_trace = go.Pie(labels=['None'], values=[100],
               hoverinfo='label+percent', textinfo='value', 
               textfont=dict(size=20), name='None',
               marker=dict(colors=colors, 
                           line=dict(color='#000000', width=2)))

main_trace = go.Pie(labels=labels, values=all_values,
               hoverinfo='label+percent', textinfo='value', 
               textfont=dict(size=20), name='Total',
               marker=dict(colors=colors, 
                           line=dict(color='#000000', width=2)))

print('All', all_values, labels)

All [12, 7, 12, 11, 11, 9, 73] ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']


In [45]:
#per decade
weekday_per_decade = {}

for t in dates:
    decade = (t.year//10)*10
    #print(decade, t.day_name())
    if decade not in weekday_per_decade:
        weekday_per_decade[decade] = []
    weekday_per_decade[decade].append(t.day_name())

#traces
traces = []
for decade in weekday_per_decade:
    c = Counter(weekday_per_decade[decade])
    weekday_per_decade[decade] = c
    
    current_values = []
    current_labels = []
    current_colors = []
    for j, l in enumerate(labels):
        if c[l] > 0:
            current_labels.append(l)
            current_values.append(c[l])
            current_colors.append(colors[j])
    
    g = go.Pie(labels=current_labels, values=current_values,
               hoverinfo='label+percent', textinfo='value', 
               textfont=dict(size=20), name=str(decade),
               marker=dict(colors=current_colors, 
                           line=dict(color='#000000', width=2)))
    traces.append(g)
    print(decade, current_values, current_labels)

traces += [main_trace]#, empty_trace]
#traces

1830 [1, 3, 2] ['Monday', 'Wednesday', 'Friday']
1840 [1, 2, 2, 1] ['Monday', 'Friday', 'Saturday', 'Sunday']
1850 [1, 1, 2, 2] ['Monday', 'Wednesday', 'Saturday', 'Sunday']
1860 [2, 2, 1] ['Tuesday', 'Thursday', 'Friday']
1870 [1, 2, 1, 2] ['Monday', 'Thursday', 'Saturday', 'Sunday']
1880 [1, 2] ['Saturday', 'Sunday']
1890 [1, 2, 2] ['Wednesday', 'Friday', 'Sunday']
1900 [1, 1, 2] ['Monday', 'Thursday', 'Sunday']
1910 [3, 1, 1] ['Monday', 'Friday', 'Sunday']
1920 [2, 1] ['Tuesday', 'Sunday']
1930 [1, 7] ['Friday', 'Sunday']
1970 [1, 3, 1, 1] ['Monday', 'Wednesday', 'Thursday', 'Sunday']
1980 [2, 2, 4, 1, 1, 9] ['Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
1990 [1, 1, 13] ['Tuesday', 'Saturday', 'Sunday']
2000 [1, 2, 1, 14] ['Monday', 'Wednesday', 'Saturday', 'Sunday']
2010 [1, 1, 1, 15] ['Monday', 'Thursday', 'Friday', 'Sunday']
2020 [1, 1] ['Monday', 'Sunday']


In [46]:
#button list

keys = list(weekday_per_decade.keys())
keys += ['Total']#, 'None']

button_list = []
for i, decade in enumerate(keys):
    #mask
    bools = [False]*len(keys)
    bools[i] = True
    if i != len(keys)-1:
        label = str(decade)
        title = 'Elections or referendums celebrated during the decade: {}'.format(decade)
    else:
        label = 'Total'
        title = 'All elections or referendums celebrated in total since 1830'
    #button
    button = {
        'label': label,
        'method': 'update',
        'args': [{'visible': bools}, 
              {'title': title}]
    }
    #add
    button_list.append(button)
    


In [47]:
updatemenus = list([
    dict(active=len(traces)-1,
         buttons=button_list,
    )
])

layout = dict(title='All elections or referendums celebrated in total since 1830', 
              showlegend=True, updatemenus=updatemenus,
             annotations=[
        dict(
            x=0.5,
            y=0,
            xshift=0,
            yshift=-40,
            visible= True,
            align='center',
            xref='paper',
            yref='paper',
            text='Checkout the GitHub <a href="https://carlosvega.github.io/Weekday_ESP_Elections/">repository</a> for more.<br>Multiple elections happening on the same date are considered as one.',
            showarrow=False,
            arrowhead=0
        )
    ])

fig = dict(data=traces, layout=layout)

iplot(fig, filename='styled_pie_chart')
plot(fig, filename='docs/styled_pie_chart.html', auto_open=False)

'docs/styled_pie_chart.html'

In [51]:
dates_fmt = [t.strftime('%Y-%m-%d') for t in dates]
weekdays = [t.day_name() for t in dates]
fig = go.Figure(data=[go.Table(header=dict(values=['Date', 'Weekday']),
                 cells=dict(values=[dates_fmt, weekdays]))
                     ])
fig.show()
plot(fig, filename='docs/styled_table.html', auto_open=False)



'docs/styled_table.html'

In [55]:
pd.DataFrame(data={'Date': dates_fmt, 'Weekday': weekdays}).to_csv('docs/processed_dates.csv')

In [ ]:
1830 [1, 3, 2] ['Monday', 'Wednesday', 'Friday']
1840 [1, 2, 2, 1] ['Monday', 'Friday', 'Saturday', 'Sunday']
1850 [1, 1, 2, 2] ['Monday', 'Wednesday', 'Saturday', 'Sunday']
1860 [2, 2, 1] ['Tuesday', 'Thursday', 'Friday']
1870 [1, 2, 1, 2] ['Monday', 'Thursday', 'Saturday', 'Sunday']
1880 [1, 2] ['Saturday', 'Sunday']
1890 [1, 2, 2] ['Wednesday', 'Friday', 'Sunday']
1900 [1, 1, 2] ['Monday', 'Thursday', 'Sunday']
1910 [3, 1, 1] ['Monday', 'Friday', 'Sunday']
1920 [2, 1] ['Tuesday', 'Sunday']
1930 [1, 7] ['Friday', 'Sunday']
1970 [1, 3, 2, 3] ['Monday', 'Wednesday', 'Thursday', 'Sunday']
1980 [6, 2, 4, 3, 1, 10] ['Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
1990 [2, 1, 21] ['Tuesday', 'Saturday', 'Sunday']
2000 [1, 3, 1, 20] ['Monday', 'Wednesday', 'Saturday', 'Sunday']
2010 [17] ['Sunday']